# Projet : CryptoBot avec Binance (OPA)

## Phase 2 : Organisation des Données

### Objectif du Document:

Ce doc a pour but de présenter les différentes étapes pour : 
- Réupérer une donnée Historique, 
- Alimenter la Base Mongo, 
- Alimenter la table SQL Histo

    TO DO
1-decision achat on part de 1000€ janvier reel 2020 - > today
2- 15min 30 min
3-

### Step 1:  Initialisation des Modules

In [2]:
# Importation des bibliothèques nécessaires
from Binance.Data import Binance_Histo as Histo
from Binance.Data import Binance_Live as live
from Binance.Dao import Drivers_MongoDB as DAO_MB
from Binance.Dao import Drivers_SQlite as DAO_SQL
from Binance.Utils import Utilitaire as util
from Binance.Utils import Technical_Analyst as util_TA
import pandas as pd
import csv

# Définition de la paire, de l'intervalle, des codes API et de la date de début
G_Symbol = 'ETHUSDT'  # La paire de devises à étudier
#G_Symbol = 'BTCUSDT'
G_Interval = "15m"  # L'intervalle de temps pour les données historiques (15 minutes dans ce cas)
G_Api_Key= "7D2zP6PkcRVmrP3KiYfePPrCeJmio8zHZKNeEtwFbHtgK6DNbWse4auahLHglgwf"  # Votre clé API Binance
G_Api_Secret= "0vZzdRUXlnoZLWYfeFi4nqFkszxnEIgw18qYlSKbEW9izxDnqKHKqugX6T5Yv129"  # Votre clé secrète API Binance
G_Start_Date="2022-12-01"  # La date de début pour les données historiques
G_Nom_BDD_SQL="qqs.db" #Le nom de la BDD

### Step 2: Chargement Fichiers Historique 

In [3]:
# Instanciation de l'objet 'L_Client' de la classe 'Binance_Histo' en utilisant les clés API et clés secrètes API préalablement déclarées
L_Client = Histo.Binance_Histo(G_Api_Key, G_Api_Secret) 

# Récupération de l'historique des prix de la paire 'G_Symbol' sur l'intervalle de temps 'G_Interval' et à partir de la date 'G_Start_Date' en utilisant l'objet 'L_Client'
L_DF_Historique_Paire = L_Client.get_data(G_Symbol, G_Interval, G_Start_Date)


In [4]:
 
L_DF_Historique_Paire.head(5)

,Symbol,Interval,Open_time,Open,High,Low,Close,Volume,Close_time,Quote_asset_volume,Nb_of_trades,Taker_buy_base_asset_volume,Taker_buy_quote_asset_volume,ignore
0,ETHUSDT,15m,1669852800000,1294.46000000,1295.82000000,1288.39000000,1293.81000000,9533.91720000,1669853699999,12325576.69518400,9611,5229.78410000,6761334.07419000,0
1,ETHUSDT,15m,1669853700000,1293.81000000,1295.88000000,1290.00000000,1291.84000000,5721.59560000,1669854599999,7398506.66031900,7626,3218.31670000,4161740.98319900,0
2,ETHUSDT,15m,1669854600000,1291.83000000,1292.82000000,1287.95000000,1288.66000000,4404.82900000,1669855499999,5681461.36079600,7246,2000.58770000,2580527.46217300,0
3,ETHUSDT,15m,1669855500000,1288.66000000,1291.30000000,1286.21000000,1290.21000000,4207.72510000,1669856399999,5422822.43608200,6167,2042.75030000,2632766.47844600,0
4,ETHUSDT,15m,1669856400000,1290.21000000,1292.74000000,1286.33000000,1287.70000000,5935.85190000,1669857299999,7652754.18607700,6695,3584.78070000,4623931.96407100,0


In [5]:
# Suppression de la colonne 'ignore'
L_DF_Historique_Paire_Update = L_DF_Historique_Paire.drop('ignore', axis=1)

# Conversion de la colonne 'Close' en flottant
L_DF_Historique_Paire_Update['Close']  = L_DF_Historique_Paire_Update['Close'].astype(float)

# Conversion de la colonne 'Open' en flottant
L_DF_Historique_Paire_Update['Open']  = L_DF_Historique_Paire_Update['Open'].astype(float)


### Step 4:Connexion à la base SQLite

In [6]:
# Instanciation d'un objet Drivers_SQLite avec le nom de la base de données
L_Connexion_SQL = DAO_SQL.Drivers_SQLite(G_Nom_BDD_SQL)

In [7]:
# Calcul des Indicateurs

L_DF_Historique_Paire_Update['IND_SMA_20'] = util_TA.Calculer_SMA(L_DF_Historique_Paire_Update['Close'], 20)

L_DF_Historique_Paire_Update['IND_SMA_30'] = util_TA.Calculer_SMA(L_DF_Historique_Paire_Update['Close'], 30)
 
L_DF_Historique_Paire_Update['IND_CHANGEPERCENT'] = util_TA.Calculer_Change_Percent(L_DF_Historique_Paire_Update['Close'])

L_DF_Historique_Paire_Update['IND_STOCH_RSI'] = util_TA.Calculer_RSI_Stochastique(L_DF_Historique_Paire_Update['Close'])
 
L_DF_Historique_Paire_Update['IND_EMA'] = util_TA.Calculer_Ema_200(L_DF_Historique_Paire_Update['Close'])
L_DF_Historique_Paire_Update['IND_RSI'] =  util_TA.Calculer_RSI(L_DF_Historique_Paire_Update['Close'])

L_DF_Historique_Paire_Update['IND_TRIX']  = util_TA.Calculer_trix(L_DF_Historique_Paire_Update['Close'])
L_DF_Historique_Paire_Update['IND_TRIX_HISTO']  = util_TA.Calculer_trix(L_DF_Historique_Paire_Update['Close'])
#conversion timestamp en date formatté
L_DF_Historique_Paire_Update['OPEN_TIME'] = L_DF_Historique_Paire_Update['Open_time'].apply(lambda x: pd.to_datetime(x, unit='ms').strftime('%Y-%m-%d %H:%M:%S'))


  
L_DF_Historique_Paire_Final = L_DF_Historique_Paire_Update[['OPEN_TIME','Symbol','Interval','Open','Close','IND_SMA_20', 'IND_SMA_30', 'Volume', 'IND_CHANGEPERCENT', 'IND_STOCH_RSI', 'IND_RSI', 'IND_TRIX','IND_TRIX_HISTO','IND_EMA']]
 
 


In [8]:
# Insérer les données de l'historique de la paire dans la table "fait_situation_histo" de la base de données
L_Resultat=L_Connexion_SQL.Alim_FaitSituation_Histo(L_DF_Historique_Paire_Final)

# Afficher le nombre de lignes insérées
print(L_Resultat, "lignes insérées")

# Fermer la connexion à la base de données
L_Connexion_SQL.CloseConnection()


7668 ligne inséré


In [15]:

 
 
# Ajout de la colonne 'Recommandation' avec les signaux d'achat/vente
L_DF_Historique_Paire_Final['Recommandation'] = util_TA.get_buy_sell_signal(L_DF_Historique_Paire_Final)
# Ajout de la colonne 'decision' avec les décisions d'achat/vente
L_DF_Historique_Paire_Final['decision'] = util_TA.verifier_decision_achat(L_DF_Historique_Paire_Final)['Decision']
# Exportation du DataFrame en fichier CSV
L_DF_Historique_Paire_Final.to_csv('aa.csv', index=False)
# Simulation du trading avec les paramètres donnés et affichage du résultat
L_Montant_Portefeuille = 10000
L_Montant_Stop_Loss=0.01         #0.1 =10%  1=100%
L_Montant_Take_Profit=0.05
res = util_TA.simulate_trading(L_DF_Historique_Paire_Final, L_Montant_Portefeuille, L_Montant_Stop_Loss,L_Montant_Take_Profit)
print("Votre montant s'eleve a :",res,"€")

g:\Mon Drive\Datascience\1-OPA\Binance\Utils\Technical_Analyst.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Signal'] = pd.Series('No Buy', index=df.index)
C:\Users\wari\AppData\Local\Temp\ipykernel_7600\3486327298.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  L_DF_Historique_Paire_Final['Recommandation'] = util_TA.get_buy_sell_signal(L_DF_Historique_Paire_Final)
C:\Users\wari\AppData\Local\Temp\ipykernel_7600\3486327298.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy o

Votre montant s'eleve a : 233472.66000000015 €
